# SCRIPT TO REMOVE INTERNAL TRANSFERS:

In [ ]:
import re
import pandas as pd
from datetime import timedelta
import os

TRANSFER_KEYWORDS = [
    "MOBILE TO", "MOBILE FROM", "ONLINE TO", "ONLINE FROM", "TRANSFER TO", "TRANSFER FROM",
    "IN-BRANCH TRANSFER", "PREAUTHORIZED TRANSFER", "TRANSFER", "WIRE REF", "WIRE TRANSFER",
    "TRANSFER TO CHECKING", "TRANSFER TO SAVINGS"
]

MASK_RE = re.compile(r'(\*{2,}\d{2,})')

def normalize_desc(s):
    if pd.isna(s): return ""
    return re.sub(r'\s+', ' ', str(s).upper()).strip()

def extract_mask(s):
    m = MASK_RE.search(s or "")
    return m.group(1) if m else None

def pair_internal_between_files(path_a: str, path_b: str, window_days: int = 3, amount_tolerance: float = 0.01):
    df_a = pd.read_csv(path_a)
    df_b = pd.read_csv(path_b)

    # track original columns to clean up later
    cols_a = df_a.columns.tolist()
    cols_b = df_b.columns.tolist()

    df_a['source'] = 'A'
    df_b['source'] = 'B'
    df = pd.concat([df_a, df_b], ignore_index=True)

    # normalize helper columns
    df['desc_norm'] = df.get('Full description', df.get('Description', '')).apply(normalize_desc)
    df['mask'] = df['desc_norm'].apply(extract_mask)
    df['abs_amount'] = df['Amount'].abs().round(2)
    df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
    df['Candidate'] = df['desc_norm'].apply(lambda d: any(k in d for k in TRANSFER_KEYWORDS))
    df['is_internal'] = False
    df['internal_id'] = pd.NA

    candidates = df[df['Candidate']].sort_values('Transaction Date').index.tolist()
    used = set()
    next_id = 1

    for idx in candidates:
        if idx in used: continue
        row = df.loc[idx]
        target_amount = row['abs_amount']
        target_sign = row['Amount'] >= 0
        window_start = row['Transaction Date'] - timedelta(days=window_days)
        window_end = row['Transaction Date'] + timedelta(days=window_days)

        pool = df.loc[
            (df.index != idx) & (~df.index.isin(used)) &
            (df['abs_amount'].sub(target_amount).abs() <= amount_tolerance) &
            (df['Transaction Date'] >= window_start) & (df['Transaction Date'] <= window_end) &
            ((df['Amount'] >= 0) != target_sign) & (df['source'] != row['source'])
        ]

        partner_idx = None
        if pd.notna(row['mask']):
            pm = pool[pool['mask'] == row['mask']]
            if not pm.empty: partner_idx = pm.index[0]

        if partner_idx is None and not pool.empty:
            partner_idx = pool.index[0]

        if partner_idx is not None:
            df.at[idx, 'is_internal'] = True
            df.at[partner_idx, 'is_internal'] = True
            df.at[idx, 'internal_id'] = next_id
            df.at[partner_idx, 'internal_id'] = next_id
            used.add(idx); used.add(partner_idx)
            next_id += 1

    # cleanup and save logic
    def save_filtered(source_label, original_path, original_cols):
        # filter for this specific file and ensure they aren't internal
        filtered = df[(df['source'] == source_label) & (df['is_internal'] == False)]
        # restore original column structure
        filtered = filtered[original_cols]

        # generate new filename
        base, ext = os.path.splitext(original_path)
        new_path = f"{base}-internals-removed{ext}"
        filtered.to_csv(new_path, index=False)
        return new_path

    path_out_a = save_filtered('A', path_a, cols_a)
    path_out_b = save_filtered('B', path_b, cols_b)

    print(f"Removed {next_id - 1} internal transfer pairs.")
    print(f"Saved: {path_out_a}\nSaved: {path_out_b}")

    return df

In [22]:
pair_internal_between_files('../path/to/primary/checking/with/internal/transfers.csv', '../path/to/secondary/checking/with/internal/transfers.csv', window_days=5)


Removed 0 internal transfer pairs.
Saved: ../gramma-statements/gramma-checking-1448-aggregated-cleaned-internals-removed-internals-removed.csv
Saved: ../gramma-statements/gramma-checking-7528-aggregated-cleaned-internals-removed-internals-removed.csv


,Transaction Date,Posted Date,Transaction Type,Check/Serial #,Full description,Merchant name,Category name,Sub-category name,Amount,Daily Posted Balance,source,desc_norm,mask,abs_amount,Candidate,is_internal,internal_id
0,2025-01-03,01/06/2025,POS,NaN,TJMAXX #0036 01-03 N. GREENVILLE SC 4081 DEBIT...,TJ Maxx,Shopping,Clothing shoes & accessories,-88.99,$691.83,A,TJMAXX #0036 01-03 N. GREENVILLE SC 4081 DEBIT...,None,88.99,False,False,<NA>
1,2025-01-03,01/06/2025,POS,NaN,OLD NAVY US 6212 01-03 GREENVILLE SC 4081 DEBI...,Old Navy,Shopping,Clothing shoes & accessories,-27.56,$691.83,A,OLD NAVY US 6212 01-03 GREENVILLE SC 4081 DEBI...,None,27.56,False,False,<NA>
2,2025-01-03,01/06/2025,POS,NaN,JIMMYS FAMILY REST 01-03 EASLEY SC 4081 DEBIT ...,Jimmys Family Rest,Dining & Entertainment,Restaurants,-33.89,$691.83,A,JIMMYS FAMILY REST 01-03 EASLEY SC 4081 DEBIT ...,None,33.89,False,False,<NA>
3,2025-01-03,01/06/2025,POS,NaN,CROSSWAY 01-03 EASLEY SC 4081 DEBIT CARD PURCHASE,Crossway,Shopping,Other shopping,-45.98,$691.83,A,CROSSWAY 01-03 EASLEY SC 4081 DEBIT CARD PURCHASE,None,45.98,False,False,<NA>
4,2025-01-04,01/06/2025,POS,NaN,ELEGANT NAILS 01-04 TAYLORS SC 4081 DEBIT CARD...,Elegant Nails,Other,Other,-55.00,$691.83,A,ELEGANT NAILS 01-04 TAYLORS SC 4081 DEBIT CARD...,None,55.00,False,False,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502,2025-12-31,12/31/2025,Debit,NaN,AchCollect ANVIS ALARM 3773 Michael D Boling A...,Anvis Alarm,Transfers & Payments,Other transfers & payments,-20.00,$2435.79,B,ACHCOLLECT ANVIS ALARM 3773 MICHAEL D BOLING A...,None,20.00,False,False,<NA>
1503,2026-01-01,01/02/2026,POS,NaN,WALGREENS STORE 2700 WADE 01-01-26 GREENVILLE ...,Walgreens,Health,Drugs & pharmacy,-48.71,$1714.09,B,WALGREENS STORE 2700 WADE 01-01-26 GREENVILLE ...,None,48.71,False,False,<NA>
1504,2026-01-01,01/02/2026,POS,NaN,APPLE.COM/BILL 01-02 866-712-7753 CA 2901 DEBI...,Apple Bill,Dining & Entertainment,Digital entertainment,-19.98,$1714.09,B,APPLE.COM/BILL 01-02 866-712-7753 CA 2901 DEBI...,None,19.98,False,False,<NA>
1505,2026-01-01,01/02/2026,ATM,NaN,WELLS FARGO BAN 01-01-26 GREENVILLE SC 2901 AT...,Wells Fargo,Cash & Checks,Cash withdrawals,-303.00,$1714.09,B,WELLS FARGO BAN 01-01-26 GREENVILLE SC 2901 AT...,None,303.00,False,False,<NA>
